In [1]:
# import libraries
import sys, os, signal, time, threading, json, webbrowser

import jupyros
import rospy, message_filters, actionlib
from std_msgs.msg import String, Header
from actionlib_msgs.msg import GoalStatus

! source /home/qcr/cgras_moveit_ws/devel/setup.bash
sys.path.append('/home/qcr/cgras_moveit_ws/devel/lib/python3/dist-packages')

# project modules
from cgras_robot.msg import RobotAction, RobotGoal

rospy.init_node('action_node')

In [ ]:
jupyros.action_client('/cgras/robot/do', RobotAction, RobotGoal, callbacks={})

In [11]:
def cb_calibrate_done(status, result):
    if status == GoalStatus.SUCCEEDED:
        print(f'success: the location apriltag found {result.data}')
    elif status == GoalStatus.ABORTED:
        print(f'error: the error is {result.error}')   

def cb_move_done(status, result):   
    if status == GoalStatus.SUCCEEDED:
        print(f'success: moved to the target')
    elif status == GoalStatus.ABORTED:
        print(f'error: the error is {result.error}')   

def cb_reset_done(status, result): 
    if status == GoalStatus.SUCCEEDED:
        print(f'success: moved to the desired pose')
    elif status == GoalStatus.ABORTED:
        print(f'error: the error is {result.error}')             

# -- callback from the GUI console
def call_robot(event, *args):
    if event == 'calibrate':
        action_client = actionlib.SimpleActionClient('/cgras/robot/do', RobotAction)
        action_client.wait_for_server()
        goal = RobotGoal(target=RobotGoal.CALIBRATE, location=[])
        action_client.send_goal(goal, done_cb=cb_calibrate_done)
            
    elif event == 'move':
        action_client = actionlib.SimpleActionClient('/cgras/robot/do', RobotAction)
        action_client.wait_for_server()
        if len(args) == 0:
            args[0] = [0, 0, 0, 0]
        goal = RobotGoal(target=RobotGoal.MOVE_TO_CELL, location=args[0])
        action_client.send_goal(goal, done_cb=cb_move_done)

    elif event == 'reset':
        action_client = actionlib.SimpleActionClient('/cgras/robot/do', RobotAction)
        action_client.wait_for_server()
        goal = RobotGoal(target=RobotGoal.MOVE_TO_HOME, location=[])
        action_client.send_goal(goal, done_cb=cb_reset_done)

# call_robot('reset')
call_robot('move', [0, 0, 0, 0,])

success: moved to the target
